In [2]:
import xarray as xr
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

In [3]:

timestamp = 1094050800
converted_time = datetime.fromtimestamp(timestamp)
print(converted_time)

2004-09-01 15:00:00


In [4]:
MIN_LON = -120
MAX_LON = -115
MIN_LAT = 31.5
MAX_LAT = 38

In [11]:
## Need to handle following error when writing code
import xarray as xr

file_path = "/root/data/rrr/integrated_weather_dataset/data/raw/Precipitation/temp_data/2010-10-31-043000.nc"
ds = xr.open_dataset(file_path, engine = 'netcdf4')


In [12]:
ds

<xarray.Dataset> Size: 28kB
Dimensions:                    (time: 1, lon: 51, lat: 66, latv: 2, lonv: 2)
Coordinates:
  * lat                        (lat) float32 264B 31.45 31.55 ... 37.85 37.95
  * latv                       (latv) int32 8B 0 1
  * lon                        (lon) float32 204B -119.9 -119.8 ... -114.9
  * lonv                       (lonv) int32 8B 0 1
  * time                       (time) object 8B 2010-10-31 04:30:00
Data variables:
    precipitationQualityIndex  (time, lon, lat) float32 13kB ...
    lat_bnds                   (lat, latv) float32 528B ...
    precipitation              (time, lon, lat) float32 13kB ...
    lon_bnds                   (lon, lonv) float32 408B ...
Attributes:
    FileHeader:                      DOI=10.5067/GPM/IMERG/3B-HH/07;\nDOIauth...
    FileInfo:                        DataFormatVersion=7e;\nTKCodeBuildVersio...
    Grid.GridHeader:                 BinMethod=ARITHMETIC_MEAN;\nRegistration...
    Grid.fullnamepath:               /Grid
    DODS_EXTRA.Unlimited_Dimension:  time
    history:                         2024-12-10 18:45:53 GMT Hyrax-1.16.3 htt...

In [35]:
file_name = file_path.split("/")[-1] 
timestamp_str = file_name.split(".")[0]
timestamp = datetime.strptime(timestamp_str, "%Y-%m-%d-%H%M%S")


In [32]:
hdf = h5py.File(file_path, 'r')

In [33]:
hdf['time'][0]

788626800

In [36]:
    
lon = hdf['lon'][:]
lat = hdf['lat'][:]
precipitation = hdf['precipitation'][0]

# Find the indices of lon and lat that are within the desired range
lon_indices = np.where((lon >= MIN_LON) & (lon <= MAX_LON))[0]
lat_indices = np.where((lat >= MIN_LAT) & (lat <= MAX_LAT))[0]

# Filter the precipitation data
precipitation_filtered = precipitation[np.ix_(lon_indices, lat_indices)]



In [37]:

# Prepare the data for saving
data_list = []
for i, lon_idx in enumerate(lon_indices):
    for j, lat_idx in enumerate(lat_indices):
        data_list.append({
            'Timestamp': timestamp,
            'Longitude': lon[lon_idx],
            'Latitude': lat[lat_idx],
            'Precipitation': precipitation_filtered[i, j]
        })

In [38]:
df = pd.DataFrame(data_list)

In [39]:
df

,Timestamp,Longitude,Latitude,Precipitation
0,2005-01-01 15:00:00,-119.949997,31.549999,0.0
1,2005-01-01 15:00:00,-119.949997,31.650000,0.0
2,2005-01-01 15:00:00,-119.949997,31.750000,0.0
3,2005-01-01 15:00:00,-119.949997,31.849998,0.0
4,2005-01-01 15:00:00,-119.949997,31.949999,0.0
...,...,...,...,...
3245,2005-01-01 15:00:00,-115.049995,37.549999,0.0
3246,2005-01-01 15:00:00,-115.049995,37.649998,0.0
3247,2005-01-01 15:00:00,-115.049995,37.750000,0.0
3248,2005-01-01 15:00:00,-115.049995,37.849998,0.0
